<a href="https://colab.research.google.com/github/bikash-bhoi/eva4/blob/master/Session4/MNIST_15K_9932_attempt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0,bias=False) #input 28 OUtput 26 RF
        self.conv2 = nn.Conv2d(16, 32, 3, padding=0,bias=False) #input 26 OUtput 24
        self.conv3 = nn.Conv2d(32, 8, 1, padding=0,bias=False) #input 24 OUtput 24
        self.pool1 = nn.MaxPool2d(2, 2)  #input 24 OUtput 12
        self.conv4 = nn.Conv2d(8, 8, 3, padding=0,bias=False) #input 12 OUtput 10
        self.conv5 = nn.Conv2d(8, 16, 3, padding=0,bias=False) #input 10 OUtput 8
        self.conv6 = nn.Conv2d(16, 32, 3, padding=0,bias=False) #input 8 OUtput 6
        self.conv7 = nn.Conv2d(32, 10, 1, padding=0,bias=False) #input 6 OUtput 6
        self.conv8 = nn.Conv2d(32, 10, 6)
        self.gap = nn.AvgPool2d(6)
        self.bn8 = nn.BatchNorm2d(8)
        self.bn16 = nn.BatchNorm2d(16)
        self.bn32 = nn.BatchNorm2d(32)
        self.bn10 = nn.BatchNorm2d(10)
        self.dropout = nn.Dropout2d(0.2)

    def forward(self, x):
        def conv_bn(conv,bn,x) : return self.dropout(bn(F.relu(conv(x))))
        x=conv_bn(self.conv1,self.bn16,x)
        x=conv_bn(self.conv2,self.bn32,x)
        x=self.pool1(self.conv3(x))  ## 1x1 conv + Maxpool
        x=conv_bn(self.conv4,self.bn8,x)
        x=conv_bn(self.conv5,self.bn16,x)
        x=self.bn32(F.relu(self.conv6(x)))
        #x = self.pool1(F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        #x = F.relu(self.conv6(F.relu(self.conv5(F.relu(self.conv4(x))))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv8(x)
        
        #x = self.conv7(x)
        #x=self.gap(x) #GAP
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=0),  #28>26
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2),
            nn.Conv2d(16, 32, 3, padding=0),  #26>24
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.Conv2d(32, 8, 1, padding=0),
            nn.MaxPool2d(2, 2)  #24>12
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=0),  #12>10
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 32, 3, padding=0), #10>8
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 8, 1, padding=0), 
            nn.Dropout(0.1),
            nn.Conv2d(8, 8, 3, padding=0), #8>6
            nn.ReLU(),
            nn.BatchNorm2d(8),
            #nn.Dropout(0.2),
            nn.Conv2d(8, 16, 3, padding=0),  #6>4
            nn.ReLU(),
            nn.BatchNorm2d(16),
            #nn.Dropout(0.2),
            nn.Conv2d(16, 10, 4)  #4>1
        )

        
    def forward(self, x):
        x = self.blk1(x)
        x = self.blk2(x)
        
        #x = self.conv7(x)
        #x=self.gap(x) #GAP
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             264
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,168
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.081, momentum=0.9)
scheduler = StepLR(optimizer, step_size=1, gamma=0.63)


for epoch in range(1, 21):
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 1 LR: [0.081]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0583, Accuracy: 9816/10000 (98%)

Epoch: 2 LR: [0.0321489]


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0237, Accuracy: 9930/10000 (99%)

Epoch: 19 LR: [1.2472555883995925e-05]


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0238, Accuracy: 9928/10000 (99%)

Epoch: 20 LR: [7.857710206917433e-06]


100%|██████████| 469/469 [00:11<00:00, 41.41it/s]



Test set: Average loss: 0.0233, Accuracy: 9930/10000 (99%)

